In [1]:
import os
import numpy as np
import numpy.linalg as la
import json as js
import _pickle as pickle
import bz2
import ray
import math
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd
import time


import torch
import torch.nn as nn
import torchvision

In [2]:
def save(file, filename, path):
    with bz2.BZ2File(os.path.join(path, filename+".pbz2"), "w") as f:
        pickle.dump(file, f)
def load(file_path):
    with bz2.BZ2File(file_path, "rb") as f:
        obj = pickle.load(f)
        return obj

def parseFloat3(struct):
    return np.asarray([struct["x"], struct["y"], struct["z"]], dtype=np.float32)
def parseFloat4(struct):
    return np.asarray([struct["value"]["x"], struct["value"]["y"], struct["value"]["z"], struct["value"]["w"]], dtype=np.float32)
def parseFloat3x3(struct):
    return np.column_stack(
        (parseFloat3(struct["c0"]), parseFloat3(struct["c1"]), parseFloat3(struct["c2"])))

In [6]:
PATH = "C:/Users/Neroro/AppData/LocalLow/DefaultCompany/Procedural Animation"
D1 = "2D_ONE_POS"
D2 = "3D_ONE_POS"
F1 = "dataset1"
F2 = "dataset2"
THREADING = True
CPUs = 24

if (THREADING):
    if (ray.is_initialized()):
        ray.shutdown()
    ray.init(num_cpus=CPUs)
    start = time.time()
"""
Parse the raw data and compute features
"""
dfs = []
for dname, dirs, files in os.walk(os.path.join(PATH, D1)):
    for fname in files:
        filePath = os.path.join(dname, fname)
        raw = js.load(open(filePath, "r"))
        if (THREADING):
            raw_obj = ray.put(raw)
            df = parse.remote(raw_obj)
            dfs.append(df)
        else:
            df = ParseRawSequence(raw)
            computeFeatures(df)
            save(df, fname.replace(".json", ""), F1)

if (THREADING):
    i = 0
    data = []
    while(len(dfs)):
        done_id, dfs = ray.wait(dfs)
        f = ray.get(done_id[0])
        data.append(f)
        # save(ray.get(done_id[0]), str(i), F1)
        i+=1
    save(data, "2D_POS", "")
    print("Done: {}".format(time.time()-start))
    ray.shutdown()

#
# for dname, dirs, files in os.walk(os.path.join(PATH, D2)):
#     for fname in files:
#         filePath = os.path.join(dname, fname)
#         raw = js.load(open(filePath, "r"))
#         df = ParseRawSequence(raw)
#         computeFeatures(df)
#         save(df, fname.replace(".json", ""), F2)

2021-03-09 12:36:12,533	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


Done: 548.6105732917786


In [6]:
raw = js.load(open(os.path.join(PATH,D1,"12.json")))

NameError: name 'PATH' is not defined

In [23]:
r = raw["frames"][0]["joints"][0]["rotMat"]
mat = parseFloat3x3(r)
print(r)
print(mat)

{'c0': {'x': 0.9872899055480957, 'y': -0.044032901525497437, 'z': 0.1527090072631836}, 'c1': {'x': 0.14798742532730103, 'y': 0.605077862739563, 'z': -0.7822918891906738}, 'c2': {'x': 0.05795420706272125, 'y': -0.7949478030204773, 'z': -0.6039036512374878}}
[[ 0.9872899   0.14798743  0.05795421]
 [-0.0440329   0.60507786 -0.7949478 ]
 [ 0.152709   -0.7822919  -0.60390365]]


In [3]:
def ParseRawSequence(clip):
    """
    In:
        clip : AnimationSequence
    Out:
        df : {  targetPos : vec3,
                frames : [
                            [   j0 : { features },
                                j1 : { features },
                                ... ,
                                jn : { features }
                            ],
                        ...,
                        ]
                        }
    """
    targetPos = clip["frames"][0]["targetsPositions"][0]
    df = {}
    df["targetPos"] = parseFloat3(targetPos)

    frames = []
    for frame in clip["frames"]:
        joints = []
        for jojo in frame["joints"]:
            jo = {}
            jo["pos"] = parseFloat3(jojo["position"])
            jo["rotEuler"] = parseFloat3(jojo["rotEuler"])
            jo["rotQuaternion"] = parseFloat4(jojo["rotQuaternion"])
            jo["rotMat"] = parseFloat3x3(jojo["rotMat"])

            jo["velocity"] = parseFloat3(jojo["velocity"])
            jo["angularVelocity"] = parseFloat3(jojo["angularVelocity"])
            jo["linearMomentum"] = parseFloat3(jojo["linearMomentum"])
            jo["angularMomentum"] = parseFloat3(jojo["angularMomentum"])

            jo["upperLimit"] = parseFloat3({"x": jojo["x"]["UpperLimit"], "y":jojo["y"]["UpperLimit"], "z":jojo["z"]["UpperLimit"]})
            jo["lowerLimit"] = parseFloat3({"x": jojo["x"]["LowerLimit"], "y":jojo["y"]["LowerLimit"], "z":jojo["z"]["LowerLimit"]})
            jo["targetValue"] = parseFloat3({"x": jojo["x"]["TargetValue"], "y":jojo["y"]["TargetValue"], "z":jojo["z"]["TargetValue"]})
            jo["currentValue"] = parseFloat3({"x": jojo["x"]["CurrentValue"], "y":jojo["y"]["CurrentValue"], "z":jojo["z"]["CurrentValue"]})
            jo["currentVelocity"] = parseFloat3({"x": jojo["x"]["CurrentVelocity"], "y":jojo["y"]["CurrentVelocity"], "z":jojo["z"]["CurrentVelocity"]})
            jo["currentAcceleration"] = parseFloat3({"x": jojo["x"]["CurrentAcceleration"], "y":jojo["y"]["CurrentAcceleration"], "z":jojo["z"]["CurrentAcceleration"]})

            jo["tGoalPos"] =  parseFloat3(jojo["tGoalPosition"])
            jo["tGoalDir"] = parseFloat3(jojo["tGoalDirection"])

            jo["posCost"] = parseFloat3(jojo["cost"]["ToTarget"])
            jo["rotCost"] = parseFloat3(jojo["cost"]["ToTargetRotation"])
            jo["targetPosition"] = parseFloat3(jojo["cost"]["TargetPosition"])
            jo["targetRotation"] = parseFloat3x3(jojo["cost"]["TargetRotation"])

            jo["contact"] = jojo["contact"]
            jo["key"] = jojo["key"]

            joints.append(jo)
        frames.append(joints)
    df["frames"] = frames
    return df

In [4]:
def computeFeatures(clip, window=12, clipSize=120):
    contactFrame = 0
    contactJoID = 0
    for i in range(len(clip["frames"])-1):
        f = clip["frames"][i]
        f2 = clip["frames"][i+1]
        for j in range(len(f)):
            jo = f[j]
            jo2 = f2[j]

            jo["posTrajectoryVec"] = jo2["pos"] - jo["pos"]    # position change
            jo["dirTrajectoryVec"] = jo2["rotEuler"] - jo["rotEuler"]   # rotation change

            if jo["key"] and jo["contact"] and contactFrame == 0:
                contactFrame = i
                contactJoID = j

            jo["phase"] = [0, 0]
            jo["tta"] = [0]

    if (contactFrame > 0):
        cycle = contactFrame / clipSize
        for i in range(window, 0, -1):
            index = contactFrame - i
            if (index < 0):
                continue
            clip["frames"][index][contactJoID]["tta"] = i
            clip["frames"][index][contactJoID]["phase"] = [1 - i/window, 0]
        for i in range(clipSize-1):
            clip["frames"][i][contactJoID]["phase"][1] = (i%cycle)/cycle * 2 * np.pi

In [5]:
@ray.remote
def parse(raw):
    df = ParseRawSequence(raw)
    computeFeatures(df)
    return pickle.dumps(df)


# Correlation analysis


In [7]:
for dname, dirs, files in os.walk(F1):
    for fname in files:
        filePath = os.path.join(F1, fname)
        data = load(filePath)
        break
    break


In [8]:
keyJo = 6
fID = np.arange(len(data["frames"]))
frames = data["frames"]

In [24]:
for k, v in data["frames"][21][keyJo].items():
    print(k, "\t:\t", v)

pos 	:	 [-0.3943923   1.2933512   0.04790461]
rotEuler 	:	 [ 14.099631 244.99411  147.33209 ]
rotQuaternion 	:	 [-0.78471446 -0.29868346  0.54078615  0.05061647]
forward 	:	 [-0.87896204 -0.2436088  -0.4099772 ]
up 	:	 [ 0.41401708 -0.81645286 -0.4024867 ]
right 	:	 [ 0.23667707  0.52350783 -0.81848884]
rotMat 	:	 [[-0.87896204  0.41401708  0.23667707]
 [-0.2436088  -0.81645286  0.52350783]
 [-0.4099772  -0.4024867  -0.81848884]]
upperLimit 	:	 [60. 60. 60.]
lowerLimit 	:	 [-60. -60. -60.]
currentValue 	:	 [0. 0. 0.]
targetValue 	:	 [-26.162346 -24.20957  -11.252455]
currentVelocity 	:	 [0. 0. 0.]
currentAcceleration 	:	 [0. 0. 0.]
tGoalPos 	:	 [-0.50074893  1.291      -0.03085136]
tGoalDir 	:	 [0. 0. 1.]
contact 	:	 False
posTrajectory 	:	 [-0.00686479 -0.00297034 -0.00233305]
dirTrajectory 	:	 [[ 0.9998596  -0.0153571  -0.00676048]
 [ 0.0156066   0.9991331   0.03861451]
 [ 0.00616236 -0.03871429  0.99923193]]
posCost 	:	 [-0.10635662 -0.00235116 -0.07875597]
phase 	:	 [0, 0]
tta 	:	 

In [9]:
posCosts = [la.norm(data["frames"][i][keyJo]["posCost"]) for i in range(len(data["frames"])-1)]
rotMatAmount = [la.norm(data["frames"][i][keyJo]["rotMat"]) for i in range(len(data["frames"])-1)]


In [10]:
fig = ex.scatter(x=fID[:-1], y=posCosts, title="Position loss")
fig.show()

In [42]:
fig = ex.line(x=fID[:-1], y=rotMatAmount, title="Rotation")
fig.show()

In [12]:
rotDiff = np.asarray([f[keyJo]["rotQuaternion"] for f in frames])
rotDiff = rotDiff[1:] - rotDiff[:-1]
rotDiff = [la.norm(r) for r in rotDiff]

fig = ex.line(x=fID[:-1], y=rotDiff, title="Rotation quaternion difference")
fig.show()

In [13]:
rotDiff = (rotDiff - np.mean(rotDiff)) / np.std(rotDiff)
posCosts = (posCosts - np.mean(posCosts)) / np.std(posCosts)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fID, y=posCosts, mode="lines", name="Distance to target"))
fig.add_trace(go.Scatter(x=fID, y=rotDiff, mode="lines", name="Rotation"))
fig.show()


In [14]:
phases = np.asarray([frames[i][keyJo]["phase"] for i in range(len(frames)-1)])
tta = np.asarray([frames[i][keyJo]["tta"] for i in range(len(frames)-1)])

In [15]:
fig = ex.bar(x=fID[:-1], y=phases[:,0])
fig.add_trace(ex.line(x=fID[:-1], y=np.cos(phases[:,1])).data[0])
fig.shows()